<a href="https://colab.research.google.com/github/ykitaguchi77/GravCont_classification_2/blob/main/Crop_unilateral_eyelid_olympia_202503.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#**Crop_unilateral_eyelid_olympia_202503**

In [1]:
# prompt: gdriveをマウント

from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [2]:
# Olympia_new
zip_path = "/content/drive/MyDrive/Deep_learning/Olympia_dataset/Olympia_new/Olympia_new_dataset.zip"

# Heltel_list
Heltel_list_path = "/content/drive/MyDrive/Deep_learning/Olympia_dataset/Olympia_new/個人情報なし　コントロールと活動期.xlsx"

# YOLO11l
model_path = "/content/drive/MyDrive/Deep_learning/Olympia_dataset/Olympia_new/295+cerebhq1-9655_yolo11l.pt"

In [3]:
# prompt: zip_pathのzipファイルを解凍

import zipfile
import os

zip_path = zip_path

# Extract the zip file
with zipfile.ZipFile(zip_path, 'r') as zip_ref:
    zip_ref.extractall("/content") # Specify the extraction path

print(f"Files extracted!")


Files extracted!


In [ ]:
# prompt: dst_dir = "content/Olympia_new/renamed_image"
# 活動期写真： IMG_0001.JPG →A0001.JPGの名前(4桁0埋め)でdst_dir内にコピー
# コントロール写真： 1.JPG →C0001.JPGの名前(4桁0埋め)でdst_dir内にコピー


import shutil
import os

active_list = "/content/Olympia_new/コントロール写真"
control_list = "/content/Olympia_new/活動期写真"
dst_dir = "/content/Olympia_new/renamed_image"

# Create the destination directory if it doesn't exist
os.makedirs(dst_dir, exist_ok=True)

# Process active images
for filename in os.listdir(active_list):
    if filename.endswith(".JPG") or filename.endswith(".jpg"):
        src_path = os.path.join(active_list, filename)
        if filename.startswith("IMG_"):
          new_filename = "A" + filename[4:].replace(".JPG",".jpg").replace(".jpg",".JPG")
        else:
          new_filename = "A" + filename.zfill(4).replace(".JPG",".jpg").replace(".jpg",".JPG")
        dst_path = os.path.join(dst_dir, new_filename)
        shutil.copy2(src_path, dst_path)  # copy2 preserves metadata
        print(f"Copied {filename} to {new_filename}")

# Process control images
for filename in os.listdir(control_list):
    if filename.endswith(".JPG") or filename.endswith(".jpg"):
        src_path = os.path.join(control_list, filename)
        new_filename = "C" + filename.zfill(4).replace(".JPG",".jpg").replace(".jpg",".JPG")
        dst_path = os.path.join(dst_dir, new_filename)
        shutil.copy2(src_path, dst_path)  # copy2 preserves metadata
        print(f"Copied {filename} to {new_filename}")


In [6]:
# prompt: # prompt: dst_dir = "content/Olympia_new/renamed_image"
# # 活動期写真： IMG_0001.JPG →A0001.JPGの名前(4桁0埋め)でdst_dir内にコピー
# # コントロール写真： 1.JPG →C0001.JPGの名前(4桁0埋め)でdst_dir内にコピー

import os
import shutil

active_list = "/content/Olympia_new/コントロール写真"
control_list = "/content/Olympia_new/活動期写真"
dst_dir = "/content/Olympia_new/renamed_image"

# Create the destination directory if it doesn't exist
if os.path.exists(dst_dir):
    shutil.rmtree(dst_dir)
os.makedirs(dst_dir, exist_ok=True)

# Process active images
for filename in os.listdir(active_list):
    if filename.lower().endswith((".jpg", ".jpeg")):  # Case-insensitive check for JPG/JPEG
        src_path = os.path.join(active_list, filename)
        name, ext = os.path.splitext(filename)
        try:
            num = int(name)
            new_filename = f"A{num:04d}{ext.upper()}"  # Use f-string for formatting
        except ValueError:
            print(f"Skipping file with non-numeric name: {filename}")
            continue
        dst_path = os.path.join(dst_dir, new_filename)
        shutil.copy2(src_path, dst_path)
        print(f"Copied {filename} to {new_filename}")

# Process control images
for filename in os.listdir(control_list):
    if filename.lower().endswith((".jpg", ".jpeg")):  # Case-insensitive check for JPG/JPEG
        src_path = os.path.join(control_list, filename)
        name, ext = os.path.splitext(filename)
        try:
            num = int(name)
            new_filename = f"C{num:04d}{ext.upper()}"  # Use f-string for formatting
        except ValueError:
            if filename.startswith("IMG_"):
                num = int(filename[4:-4])
                new_filename = f"C{num:04d}{ext.upper()}"
            else:
                print(f"Skipping file with non-numeric name: {filename}")
                continue
        dst_path = os.path.join(dst_dir, new_filename)
        shutil.copy2(src_path, dst_path)
        print(f"Copied {filename} to {new_filename}")


Copied 117.JPG to A0117.JPG
Copied 269.JPG to A0269.JPG
Copied 119.JPG to A0119.JPG
Copied 662.JPG to A0662.JPG
Copied 545.JPG to A0545.JPG
Copied 474.JPG to A0474.JPG
Copied 192.JPG to A0192.JPG
Copied 569.JPG to A0569.JPG
Copied 691.JPG to A0691.JPG
Copied 927.JPG to A0927.JPG
Copied 819.JPG to A0819.JPG
Copied 1022.JPG to A1022.JPG
Copied 887.JPG to A0887.JPG
Copied 1026.JPG to A1026.JPG
Copied 748.JPG to A0748.JPG
Copied 1025.JPG to A1025.JPG
Copied 1014.JPG to A1014.JPG
Copied 230.JPG to A0230.JPG
Copied 31.JPG to A0031.JPG
Copied 538.JPG to A0538.JPG
Copied 768.JPG to A0768.JPG
Copied 439.JPG to A0439.JPG
Copied 977.JPG to A0977.JPG
Copied 451.JPG to A0451.JPG
Copied 814.JPG to A0814.JPG
Copied 223.JPG to A0223.JPG
Copied 29.JPG to A0029.JPG
Copied 268.JPG to A0268.JPG
Copied 96.JPG to A0096.JPG
Copied 232.JPG to A0232.JPG
Copied 322.JPG to A0322.JPG
Copied 963.JPG to A0963.JPG
Copied 737.JPG to A0737.JPG
Copied 778.JPG to A0778.JPG
Copied 204.JPG to A0204.JPG
Copied 404.JPG to A